In [1]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!nvidia-smi

Fri Dec 11 11:32:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Clone OpenAI-Gym

In [2]:
!git clone https://github.com/openai/gym.git

Cloning into 'gym'...
remote: Enumerating objects: 10828, done.
remote: Total 10828 (delta 0), reused 0 (delta 0), pack-reused 10828
Receiving objects: 100% (10828/10828), 4.14 MiB | 6.40 MiB/s, done.
Resolving deltas: 100% (7332/7332), done.


Installing requirements for the Gym

In [3]:
%cd gym
!pip3 install -e .

/content/gym
Obtaining file:///content/gym
  Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3:
      Successfully uninstalled gym-0.17.3
  Running setup.py develop for gym


Checking whether the installation was successful by listing all the available environments

In [ ]:
from gym import envs
envids = [spec.id for spec in envs.registry.all()]
for envid in sorted(envids):
    print(envid)

In [5]:
%cd ..

/content


Cloning the DT Gym and our repository

In [ ]:
!git clone https://github.com/Comicboy/deep-rl-for-the-duckietown-aido_lf-challenge

In [7]:
!git clone https://github.com/duckietown/gym-duckietown.git

Cloning into 'gym-duckietown'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 5528 (delta 95), reused 160 (delta 61), pack-reused 5322
Receiving objects: 100% (5528/5528), 79.04 MiB | 22.98 MiB/s, done.
Resolving deltas: 100% (3204/3204), done.


In [8]:
%cd gym-duckietown

/content/gym-duckietown


Going back to an older commit because it is possible only with this version to register the environments

In [ ]:
!git checkout dcf8dd3

In [ ]:
!pip install -e .

In [11]:
%cd ..

/content


Copy our map into the maps folder

In [12]:
!cp deep-rl-for-the-duckietown-aido_lf-challenge/my.yaml gym-duckietown/gym_duckietown/maps

Import Gym

In [13]:
import gym

Import the DT Gym

In [14]:
%cd gym-duckietown/

/content/gym-duckietown


In [15]:
import gym_duckietown.envs.duckietown_env

INFO:gym-duckietown:gym-duckietown 2018.10.1

INFO:gym-duckietown:Registering gym environment id: Duckietown-small_loop-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_dyn_duckiebots-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-my-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_empty-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-straight_road-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_pedestrians-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-4way-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-zigzag_dists-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-loop_obstacles-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-small_loop_cw-v0
INFO:gym-duckietown:Registering gym environment id: Duckietown-udem1-v0


Get the StableBaselines

In [ ]:
!apt-get update

In [ ]:
!apt-get install cmake libopenmpi-dev python3-dev zlib1g-dev

In [ ]:
!pip install stable-baselines[mpi]

In [ ]:
!pip install tensorflow==1.15.0

In [20]:
import numpy as np
from stable_baselines import PPO2
from stable_baselines.common.evaluation import evaluate_policy

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Get Optuna

In [ ]:
!pip install optuna

In [22]:
import optuna

Virtual display is necessary for training

In [ ]:
!apt update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
30 packages can be upgraded. Run '

In [ ]:
!apt-get install python-opengl -y
!apt-get install xvfb -y

In [ ]:
!pip install pyvirtualdisplay
!pip install piglet

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

ModuleNotFoundError: ignored

A different way of creating a virtual display

In [ ]:
!apt update

In [ ]:
!apt-get install ffmpeg freeglut3-dev xvfb  # For visualization

In [25]:
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

Check if the DT environments follow the Gym interface

In [ ]:
from gym_duckietown.simulator import Simulator

In [ ]:
from stable_baselines.common.env_checker import check_env
check_env(Simulator())

In [ ]:
from gym_duckietown.envs.duckietown_env import DuckietownEnv
check_env(DuckietownEnv())

In [ ]:
env = gym.make('Duckietown-straight_road-v0')

In [ ]:
env.reset()

In [ ]:
env.render()

Creating a PPO model

In [30]:
model = PPO2('CnnPolicy', env, verbose=1).learn(1000)

Wrapping the env in a DummyVecEnv.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



DEBUG:gym-duckietown:No tile found at [-0.00573276  0.          0.21150281] (-1, 0)
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [0.0841755  0.         0.17378338]
DEBUG:gym-duckietown:l_pos: [-0.00573276  0.          0.21150281]
DEBUG:gym-duckietown:r_pos: [0.17408376 0.         0.13606395]
DEBUG:gym-duckietown:f_pos: [0.03891219 0.         0.06589348]
DEBUG:gym-duckietown:No tile found at [-0.09499647  0.          0.1887226 ] (-1, 0)
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [0.0024561  0.         0.19176342]
DEBUG:gym-duckietown:l_pos: [-0.09499647  0.          0.1887226 ]
DEBUG:gym-duckietown:r_pos: [0.09990867 0.         0.19480425]
DEBUG:gym-duckietown:f_pos: [0.00610509 0.    

-------------------------------------
| approxkl           | 0.008508054  |
| clipfrac           | 0.1171875    |
| explained_variance | 0.0165       |
| fps                | 9            |
| n_updates          | 1            |
| policy_entropy     | 2.837685     |
| policy_loss        | -0.008219914 |
| serial_timesteps   | 128          |
| time_elapsed       | 2.26e-05     |
| total_timesteps    | 128          |
| value_loss         | 63.20819     |
-------------------------------------


INFO:gym-duckietown:Pos: [0.24088278 0.         0.4626493 ] angle 0.3309380677000357
INFO:gym-duckietown:Pos: [0.20305326 0.         0.47564651] angle 0.3309380677000357
INFO:gym-duckietown:Pos: [0.16522374 0.         0.48864373] angle 0.3309380677000357
INFO:gym-duckietown:Pos: [0.12739422 0.         0.50164094] angle 0.3309380677000357
INFO:gym-duckietown:Pos: [0.1071949  0.         0.50844517] angle 0.3188945778222252
DEBUG:gym-duckietown:No tile found at [0.1079187  0.         0.58718828] (0, 1)
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.0
DEBUG:gym-duckietown:pos: [0.08440492 0.         0.51596958]
DEBUG:gym-duckietown:l_pos: [0.06089114 0.         0.44475089]
DEBUG:gym-duckietown:r_pos: [0.1079187  0.         0.58718828]
DEBUG:gym-duckietown:f_pos: [0.16986735 0.         0.48775305]
INFO:gym-duckietown:Stopping the simulator because we are at an invalid pose.
INFO:gym-duckietown:Starting at [0.50152825 0. 

--------------------------------------
| approxkl           | 0.00087840355 |
| clipfrac           | 0.0           |
| explained_variance | 0.00108       |
| fps                | 18            |
| n_updates          | 2             |
| policy_entropy     | 2.8376646     |
| policy_loss        | 0.00036497996 |
| serial_timesteps   | 256           |
| time_elapsed       | 13.9          |
| total_timesteps    | 256           |
| value_loss         | 79465.08      |
--------------------------------------


...

--------------------------------------
| approxkl           | 0.0032839386  |
| clipfrac           | 0.03515625    |
| explained_variance | -0.0258       |
| fps                | 19            |
| n_updates          | 5             |
| policy_entropy     | 2.8353589     |
| policy_loss        | -0.0036225286 |
| serial_timesteps   | 640           |
| time_elapsed       | 33.9          |
| total_timesteps    | 640           |
| value_loss         | 37.993126     |
--------------------------------------


INFO:gym-duckietown:Pos: [1.21815606 0.         0.27139196] angle 2.7812272794502535
INFO:gym-duckietown:Pos: [1.20372109 0.         0.26556511] angle 2.7346398114791377
INFO:gym-duckietown:Pos: [1.23188636 0.         0.27722631] angle 2.7634650453242564
INFO:gym-duckietown:Pos: [1.26906066 0.         0.29199355] angle 2.7634650453242564
INFO:gym-duckietown:Pos: [1.27420924 0.         0.29408617] angle 2.747620096715364
INFO:gym-duckietown:Pos: [1.3111449  0.         0.30944055] angle 2.747620096715364
INFO:gym-duckietown:Pos: [1.34024056 0.         0.32233859] angle 2.7010326287442483
INFO:gym-duckietown:Pos: [1.35084222 0.         0.32764215] angle 2.6544451607731325
INFO:gym-duckietown:Pos: [1.37930718 0.         0.34358077] angle 2.6078576928020167
INFO:gym-duckietown:Pos: [1.37068712 0.         0.33858309] angle 2.624500198208739
INFO:gym-duckietown:Pos: [1.37621594 0.         0.34161982] angle 2.654145221536915
INFO:gym-duckietown:Pos: [1.34087469 0.         0.32288493] angle 2.6

-------------------------------------
| approxkl           | 0.004223713  |
| clipfrac           | 0.0390625    |
| explained_variance | 0.00373      |
| fps                | 19           |
| n_updates          | 6            |
| policy_entropy     | 2.8384414    |
| policy_loss        | -0.006971908 |
| serial_timesteps   | 768          |
| time_elapsed       | 40.6         |
| total_timesteps    | 768          |
| value_loss         | 34762.047    |
-------------------------------------


INFO:gym-duckietown:Pos: [0.74881465 0.         0.37698061] angle 2.5366909152071164
INFO:gym-duckietown:Pos: [0.71591233 0.         0.35423336] angle 2.5366909152071164
INFO:gym-duckietown:Pos: [0.68301001 0.         0.33148611] angle 2.5366909152071164
INFO:gym-duckietown:Pos: [0.65286675 0.         0.31054802] angle 2.5322831673456423
INFO:gym-duckietown:Pos: [0.65798272 0.         0.31427318] angle 2.4921701299972376
INFO:gym-duckietown:Pos: [0.62612539 0.         0.29008412] angle 2.4921701299972376
INFO:gym-duckietown:Pos: [0.62003769 0.         0.28530259] angle 2.4594154991680175
INFO:gym-duckietown:Pos: [0.62440485 0.         0.28902203] angle 2.4128280311969017
INFO:gym-duckietown:Pos: [0.65183615 0.         0.3143447 ] angle 2.3794605485479186
INFO:gym-duckietown:Pos: [0.66931115 0.         0.33093769] angle 2.3846968165914366
INFO:gym-duckietown:Pos: [0.68977804 0.         0.35119247] angle 2.338109348620321
INFO:gym-duckietown:Pos: [0.68900166 0.         0.35035736] angle 

-------------------------------------
| approxkl           | 0.0072480636 |
| clipfrac           | 0.0859375    |
| explained_variance | -0.00121     |
| fps                | 18           |
| n_updates          | 7            |
| policy_entropy     | 2.8401198    |
| policy_loss        | -0.016371448 |
| serial_timesteps   | 896          |
| time_elapsed       | 47.1         |
| total_timesteps    | 896          |
| value_loss         | 71509.05     |
-------------------------------------


In [ ]:
evaluate_policy(model = model, env = env, n_eval_episodes = 1)

Defining the parameters for hyperparameter optimization.

The ranges for the parameters are based on this article:
https://medium.com/aureliantactics/ppo-hyperparameters-and-ranges-6fc2d29bccbe

The suggestions' distributions where given intuitively.

(I have no idea if 'max_grad_norm' is necessary at all here.)

In [ ]:
def ppo_hyperopt(trial):

  return {
      'n_steps': int(trial.suggest_int('n_steps', 32, 5000)),
      'nminibatches': trial.suggest_int('nminibatches', 4, 4096),
      'gamma': trial.suggest_uniform('gamma', 0.8, 0.9997),
      'learning_rate': trial.suggest_loguniform('learning_rate', 5e-6, 0.003),
      'ent_coef': trial.suggest_uniform('ent_coef', 0, 0.01),
      'noptepochs': int(trial.suggest_int('noptepochs', 3, 30)),
      'vf_coef': trial.suggest_uniform('vf_coef', 0.5, 1),
      'lam': trial.suggest_uniform('lam', 0.9, 1),
      'cliprange': trial.suggest_uniform('cliprange', 0.1, 0.3),
      #'max_grad_norm': trial.suggest_loguniform('max_grad_norm', 0.1, 0.9),
  }

Own trials.

In [ ]:
def ppo_hyperopt(trial):

  return {
      #'n_steps': int(trial.suggest_discrete_uniform('n_steps', 16, 1024, 4)),
      'gamma': trial.suggest_loguniform('gamma', 0.8, 0.95),
      'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-2),
      'ent_coef': trial.suggest_loguniform('ent_coef', 1e-3, 1e-1),
      'noptepochs': int(trial.suggest_int('noptepochs', 3, 10)),
      #'vf_coef': trial.suggest_loguniform('vf_coef', 0.1, 0.9),
      #'max_grad_norm': trial.suggest_loguniform('max_grad_norm', 0.1, 0.9),
      #'lam': trial.suggest_loguniform('lam', 0.8, 0.99),
      #'cliprange': trial.suggest_loguniform('cliprange', 0.15, 0.25)
  }

In [ ]:
def optimize_agent(trial):
  model_params = ppo_hyperopt(trial)
  env = gym.make('Duckietown-straight_road-v0')
  model = PPO2('CnnPolicy', env, verbose=1, **model_params)

  # Learning
  model.learn(100000)

  # Evaluation
  rewards = []
  n_episodes = 0
  reward_sum = 0.0

  obsv = env.reset()
  while n_episodes < 1000:
    action, _ = model.predict(obsv)
    obsv, reward, done, info = env.step(action)
    reward_sum = reward_sum + reward

    if done:
      rewards.append(reward_sum)
      reward_sum = 0.0
      n_episodes = n_episodes + 1
      obsv = env.reset()

  last_reward = np.mean(rewards)
  trial.report(-1 * last_reward, n_episodes)

  if trial.should_prune():
    raise optuna.TrialPruned()

  return -1 * last_reward

#env.close()

#mean_reward, std_reward = evaluate_policy(model = model, env = env, n_eval_episodes = 10)
#return mean_reward

In [ ]:
from optuna.samplers import TPESampler
sampler = TPESampler(seed = 3)

In [ ]:
study = optuna.create_study(pruner = optuna.pruners.MedianPruner(n_startup_trials = 5, n_warmup_steps = 10))

[I 2020-12-10 20:53:06,627] A new study created in memory with name: no-name-035d2ceb-6c6a-481e-932a-06261c2ad448


In [ ]:
study.optimize(optimize_agent, n_trials = 20, gc_after_trial = True, catch = (float('nan'), ))
print('study.best_params')

DEBUG:gym-duckietown:Falling back to non-multisampled frame buffer
DEBUG:gym-duckietown:Falling back to non-multisampled frame buffer
DEBUG:gym-duckietown:loading map file "/content/gym-duckietown/gym_duckietown/maps/straight_road.yaml"
DEBUG:gym-duckietown:loading mesh "/content/gym-duckietown/gym_duckietown/meshes/duckiebot.obj"
DEBUG:gym-duckietown:loading materials from "/content/gym-duckietown/gym_duckietown/meshes/duckiebot.mtl"
DEBUG:gym-duckietown:loading texture "/content/gym-duckietown/gym_duckietown/textures/straight_1.png"
DEBUG:gym-duckietown:No tile found at [0.50411701 0.         0.58680606] (0, 1)
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [0.50411701 0.         0.58680606]
DEBUG:gym-duckietown:l_pos: [0.46964266 0.         0.67800786]
DEBUG:gym-duckietown:r_pos: [0.53859135 0.         0.49560425]
DEBUG:gym-duckietown:f_pos: [0.39467484 0.         0.54543684]
INFO:gym-

Wrapping the env in a DummyVecEnv.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



DEBUG:gym-duckietown:No tile found at [ 0.54212483  0.         -0.08351366] (0, -1)
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [0.52267855 0.         0.03185896]
DEBUG:gym-duckietown:l_pos: [0.4265347  0.         0.01565372]
DEBUG:gym-duckietown:r_pos: [0.6188224 0.        0.0480642]
DEBUG:gym-duckietown:f_pos: [ 0.54212483  0.         -0.08351366]
INFO:gym-duckietown:Starting at [0.11005996 0.         0.36710717] 6.238409608195975
INFO:gym-duckietown:Pos: [0.11005996 0.         0.36710717] angle 6.238409608195975
INFO:gym-duckietown:Pos: [0.07582018 0.         0.36549667] angle 6.2339585206212975
INFO:gym-duckietown:Pos: [0.11112291 0.         0.36748635] angle 6.2198099357529575
INFO:gym-duckietown:Pos: [0.14765104 0.         0.37025948] angle 6.195016208465557
INFO:gym-duckietown:Pos

Wrapping the env in a DummyVecEnv.


DEBUG:gym-duckietown:No tile found at [0.41014242 0.         0.59739379] (0, 1)
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [0.49947005 0.         0.55831917]
DEBUG:gym-duckietown:l_pos: [0.41014242 0.         0.59739379]
DEBUG:gym-duckietown:r_pos: [0.58879767 0.         0.51924455]
DEBUG:gym-duckietown:f_pos: [0.4525805  0.         0.45112602]
DEBUG:gym-duckietown:No tile found at [ 0.00927344  0.         -0.08112455] (0, -1)
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.3
DEBUG:gym-duckietown:pos: [0.00652552 0.         0.01633672]
DEBUG:gym-duckietown:l_pos: [ 0.00927344  0.         -0.08112455]
DEBUG:gym-duckietown:r_pos: [0.0037776  0.         0.11379799]
DEBUG:gym-duckietown:f_pos: [0.12347904 0.         0.01963423]
INFO:gym-duckietown:Starting at [0.48689984 0.         0.49457734] 2.2167299029503367
INFO:gy

...

Wrapping the env in a DummyVecEnv.


INFO:gym-duckietown:Starting at [0.26285409 0.         0.40432862] 3.591879746883685
INFO:gym-duckietown:Pos: [0.26285409 0.         0.40432862] angle 3.591879746883685
INFO:gym-duckietown:Pos: [0.22815459 0.         0.42029536] angle 3.5538196966186857
INFO:gym-duckietown:Pos: [0.19222302 0.         0.43630778] angle 3.567804323449199
INFO:gym-duckietown:Pos: [0.18912445 0.         0.4377725 ] angle 3.59853616173683
INFO:gym-duckietown:Pos: [0.22502066 0.         0.42012421] angle 3.59853616173683
INFO:gym-duckietown:Pos: [0.2096752  0.         0.42811785] angle 3.6451236297079457
INFO:gym-duckietown:Pos: [0.17463983 0.         0.4474187 ] angle 3.6451236297079457
INFO:gym-duckietown:Pos: [0.17339487 0.         0.44808049] angle 3.6152355921478803
INFO:gym-duckietown:Pos: [0.17122828 0.         0.4492555 ] angle 3.661823060118996
INFO:gym-duckietown:Pos: [0.14333899 0.         0.46610705] angle 3.708410528090112
INFO:gym-duckietown:Pos: [0.10959444 0.         0.48758507] angle 3.70841

Wrapping the env in a DummyVecEnv.


INFO:gym-duckietown:Starting at [0.11284758 0.         0.23745811] 3.4368120665813824
INFO:gym-duckietown:Pos: [0.11284758 0.         0.23745811] angle 3.4368120665813824
INFO:gym-duckietown:Pos: [0.07457805 0.         0.2490961 ] angle 3.4368120665813824
INFO:gym-duckietown:Pos: [0.03630851 0.         0.26073409] angle 3.4368120665813824
DEBUG:gym-duckietown:No tile found at [-0.02683622  0.          0.27993678] (-1, 0)
DEBUG:gym-duckietown:Invalid pose. Collision free: True On drivable area: False
DEBUG:gym-duckietown:safety_factor: 1.0
DEBUG:gym-duckietown:pos: [0.05927023 0.         0.2537513 ]
DEBUG:gym-duckietown:l_pos: [0.08109147 0.         0.32550667]
DEBUG:gym-duckietown:r_pos: [0.037449   0.         0.18199592]
DEBUG:gym-duckietown:f_pos: [-0.02683622  0.          0.27993678]
INFO:gym-duckietown:Stopping the simulator because we are at an invalid pose.
INFO:gym-duckietown:Starting at [0.16378347 0.         0.47099545] 4.100357071588675
INFO:gym-duckietown:Pos: [0.16378347 0.

KeyboardInterrupt: ignored

Save study

In [ ]:
joblib.dump(study, 'stdy_ppo.pkl')

Resume study

In [ ]:
study = joblib.load('study_ppo.pkl')
print('Best trial until now:')
print(' Value: ', study.best_trial.value)
print(' Params: ')
for key, value in study.best_trial.params.items():
    print(f'    {key}: {value}')

This is our CNN structure using tensorflow.nn.

See this (and cry):

https://stable-baselines.readthedocs.io/en/master/_modules/stable_baselines/common/policies.html

In [ ]:
from stable_baselines.common.tf_util import batch_to_seq, seq_to_batch
from stable_baselines.common.tf_layers import conv, linear, conv_to_fc, lstm
from stable_baselines.common.distributions import make_proba_dist_type, CategoricalProbabilityDistribution, \
    MultiCategoricalProbabilityDistribution, DiagGaussianProbabilityDistribution, BernoulliProbabilityDistribution
from stable_baselines.common.input import observation_input

In [ ]:
def my_cnn(scaled_images, **kwargs):

    activ = tf.nn.leaky_relu
    layer_1 = activ(conv(scaled_images, n_filters=64, filter_size=5, stride=2, init_scale=np.sqrt(2), **kwargs))
    layer_2 = activ(conv(layer_1, n_filters=64, filter_size=3, stride=2, init_scale=np.sqrt(2), **kwargs))
    max_pool_1 = max_pool2d(layer_2, ksize = 2, strides = 2)
    layer_3 = activ(conv(max_pool_1, n_filters=128, filter_size=3, stride=2, init_scale=np.sqrt(2), **kwargs))
    layer_4 = activ(conv(layer_3, n_filters=128, filter_size=3, stride=2, init_scale=np.sqrt(2), **kwargs))
    max_pool_2= max_pool2d(layer_4, ksize = 2, strides = 2)
    layer_5 = activ(conv(max_pool_2, n_filters=256, filter_size=3, stride=2, init_scale=np.sqrt(2), **kwargs))
    max_pool_3= max_pool2d(layer_5, ksize = 2, strides = 2)

    max_pool_3 = conv_to_fc(max_pool_3) # no flatten???

    linear_ 1 = activ(linear(max_pool_3, n_hidden = 2048, init_scale=np.sqrt(2)))
    linear_ 2 = activ(linear(linear_1, n_hidden = 2048, init_scale=np.sqrt(2)))
    return activ(linear(layer_3, n_hidden = 1, init_scale=np.sqrt(2)))
